# Data-Driven Assessment of Canadian Crops and Prices for US Venture Planning

### Objectives:

The goal of this project is to support strategic decisions for a US-based venture capital firm interested in the craft beer and spirits supply chain by analyzing Canadian agricultural data—specifically crop production and farm prices—while evaluating exchange rate impacts using R and SQLite.

The first task is to provide a high level analysis of crop production in Canada. The stakeholders want to understand the current and historical performance of certain crop types in terms of supply and price. For now they are mainly interested in a macro-view of Canada’s crop farming industry, and how it relates to the relative value of the Canadian and US dollars.

We will be asked questions that will help us understand the data. So, we will also be asked to create four tables in database, and load the tables using the provided datasets from R using the RODBC/RSQLite package. 



## I. Understanding the Datasets

The purpose is to familiarize with 4 datasets from Statistics Canada and the Bank of Canada.

The datasets Used are:
- Annual_Crop_Data.csv: Annual crop production metrics.
- Monthly_Farm_Prices.csv: Monthly crop price data.
- Daily_FX.csv: Daily USD-CAD exchange rates.
- Monthly_FX.csv: Monthly average USD-CAD exchange rates.

There are clarity on data structure, column names, and data types—crucial for correct table creation and querying.

## II. Creating & Loading the Dataset Tables

The purpose of this step is to load CSV datasets into SQLite using the RSQLite and DBI packages:
- Connect to SQLite DB
- Create Tables CROP_DATA; FARM_PRICES; DAILY_FX; MONTHLY_FX.
- Load DataFrames from CSV and write to tables using dbWriteTable.
- Use dbListTables(con) to confirm successful table creation and loading.

In [31]:
# Install necessary packages 
install.packages("RSQLite")
install.packages("DBI")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [32]:
# Load the libraries
library(RSQLite)
library(DBI)

In [33]:
# Connect to (or create) a SQLite database
con <- dbConnect(RSQLite::SQLite(), "crop_analysis.db")

### 1. Create Tables in SQLite

In [36]:
# Create CROP_DATA Table
df1 <- dbExecute(conn, "
  CREATE TABLE CROP_DATA (
    CD_ID INTEGER NOT NULL,
    YEAR DATE NOT NULL,
    CROP_TYPE VARCHAR(20) NOT NULL,
    GEO VARCHAR(20) NOT NULL,
    SEEDED_AREA INTEGER NOT NULL,
    HARVESTED_AREA INTEGER NOT NULL,
    PRODUCTION INTEGER NOT NULL,
    AVG_YIELD INTEGER NOT NULL,
    PRIMARY KEY (CD_ID)
  )
")

# Check if table was created successfully
if (df1 == -1){
  cat("An error has occurred.\n")
  msg <- odbcGetErrMsg(conn)
  print(msg)
} else {
  cat("Table was created successfully.\n")
}

Table was created successfully.


In [37]:
# Create FARM_PRICES Table
df2 <- dbExecute(conn, "
  CREATE TABLE FARM_PRICES (
    FP_ID INTEGER NOT NULL,
    DATE DATE NOT NULL,
    GEO VARCHAR(20) NOT NULL,
    CROP_TYPE VARCHAR(20) NOT NULL,
    PRICE DECIMAL(10,2) NOT NULL,
    PRIMARY KEY (FP_ID)
  )
")

if (df2 == -1){
  cat('Error creating FARM_PRICES table.\n')
} else {
  cat("FARM_PRICES table created successfully.\n")
}

FARM_PRICES table created successfully.


In [38]:
# Create DAILY_FX Table
df3 <- dbExecute(conn, "
  CREATE TABLE DAILY_FX (
    FXD_ID INTEGER NOT NULL,
    DATE DATE NOT NULL,
    USD_CAD DECIMAL(10,4) NOT NULL,
    PRIMARY KEY (FXD_ID)
  )
")

if (df3 == -1){
  cat('Error creating DAILY_FX table.\n')
} else {
  cat("DAILY_FX table created successfully.\n")
}

DAILY_FX table created successfully.


In [41]:
df4 <- dbExecute(conn, "
  CREATE TABLE MONTHLY_FX (
    FXM_ID INTEGER NOT NULL,
    DATE DATE NOT NULL,
    USD_CAD DECIMAL(10,4) NOT NULL,
    PRIMARY KEY (FXM_ID)
  )
")

if (df4 == -1){
  cat('Error creating MONTHLY_FX table.\n')
} else {
  cat("MONTHLY_FX table created successfully.\n")
}

MONTHLY_FX table created successfully.


### 2. Load the Dataframes into The SQLite Database Tables

In [42]:
# Load each CSV file into a data frame
crop_data <- read.csv("/Annual_Crop_Data.csv")
farm_prices <- read.csv("/Monthly_Farm_Prices.csv")
daily_fx <- read.csv("/Daily_FX.csv")
monthly_fx <- read.csv("/Monthly_FX.csv")

In [48]:
head(crop_data)

,CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD
,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>
1,0,1965-12-31,Barley,Alberta,1372000,1372000,2504000,1825
2,1,1965-12-31,Barley,Canada,2476800,2476800,4752900,1920
3,2,1965-12-31,Barley,Saskatchewan,708000,708000,1415000,2000
4,3,1965-12-31,Canola,Alberta,297400,297400,215500,725
5,4,1965-12-31,Canola,Canada,580700,580700,512600,885
6,5,1965-12-31,Canola,Saskatchewan,224600,224600,242700,1080


In [49]:
head(farm_prices)

,CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
,<int>,<chr>,<chr>,<chr>,<dbl>
1,0,1985-01-01,Barley,Alberta,127.39
2,1,1985-01-01,Barley,Saskatchewan,121.38
3,2,1985-01-01,Canola,Alberta,342.00
4,3,1985-01-01,Canola,Saskatchewan,339.82
5,4,1985-01-01,Rye,Alberta,100.77
6,5,1985-01-01,Rye,Saskatchewan,109.75


In [50]:
head(daily_fx)

,DFX_ID,DATE,FXUSDCAD
,<int>,<chr>,<dbl>
1,0,2017-01-03,1.3435
2,1,2017-01-04,1.3315
3,2,2017-01-05,1.3244
4,3,2017-01-06,1.3214
5,4,2017-01-09,1.3240
6,5,2017-01-10,1.3213


In [51]:
head(monthly_fx)

,DFX_ID,DATE,FXUSDCAD
,<int>,<chr>,<dbl>
1,0,2017-01-01,1.319276
2,1,2017-02-01,1.310726
3,2,2017-03-01,1.338643
4,3,2017-04-01,1.344021
5,4,2017-05-01,1.360705
6,5,2017-06-01,1.329805


### 3. Write the Dataframes to The SQLite Database Tables

In [43]:
# Write data frames to the database as tables
dbWriteTable(con, "CROP_DATA", crop_data, overwrite = TRUE)
dbWriteTable(con, "FARM_PRICES", farm_prices, overwrite = TRUE)
dbWriteTable(con, "DAILY_FX", daily_fx, overwrite = TRUE)
dbWriteTable(con, "MONTHLY_FX", monthly_fx, overwrite = TRUE)

In [44]:
# check List of Tables in the present db
dbListTables(con)

[1] "CROP_DATA"   "DAILY_FX"    "FARM_PRICES" "MONTHLY_FX"

## III. SQL Querying Using RSQLite

The purpose of this step is to answer business questions using SQL queries in R via dbGetQuery().

1. How many records are in the farm prices dataset?

In [56]:
# Query to count number of records in FARM_PRICES table
result3 <-dbGetQuery(con, 'SELECT COUNT(*) AS total_records FROM FARM_PRICES')
result3

total_records
<int>
2678


2. Which provinces are included in the farm prices dataset?

In [55]:
# Query to list distinct provinces in the FARM_PRICES dataset
result4 <- dbGetQuery(con, 'SELECT DISTINCT GEO AS province FROM FARM_PRICES')
result4

province
<chr>
Alberta
Saskatchewan


3. How many hectares of Rye were harvested in Canada in 1968?

In [62]:
result5 <- dbGetQuery(con, "
  SELECT HARVESTED_AREA 
  FROM CROP_DATA 
  WHERE CROP_TYPE = 'Rye' 
    AND GEO = 'Canada' 
    AND strftime('%Y', YEAR) = '1968';
")
result5

HARVESTED_AREA
<int>
274100


4. Query and display the first 6 rows of the farm prices table for Rye.

In [65]:
# SQL query to retrieve the first 6 rows for Rye
result6 <- dbGetQuery(con, "
  SELECT * 
  FROM FARM_PRICES 
  WHERE CROP_TYPE = 'Rye' 
  LIMIT 6;
")

result6

CD_ID,DATE,CROP_TYPE,GEO,PRICE_PRERMT
<int>,<chr>,<chr>,<chr>,<dbl>
4,1985-01-01,Rye,Alberta,100.77
5,1985-01-01,Rye,Saskatchewan,109.75
10,1985-02-01,Rye,Alberta,95.05
11,1985-02-01,Rye,Saskatchewan,103.46
16,1985-03-01,Rye,Alberta,96.77
17,1985-03-01,Rye,Saskatchewan,106.38


5. Which provinces grew Barley?

In [67]:
# Query to find distinct provinces that grew Barley
result5 <- dbGetQuery(con, "
  SELECT DISTINCT GEO 
  FROM CROP_DATA 
  WHERE CROP_TYPE = 'Barley';
")

# Display the result
print(result5)

           GEO
1      Alberta
2       Canada
3 Saskatchewan


6. Find the first and last dates for the farm prices data.

In [69]:
# Query to find the first and last date in the FARM_PRICES table
result_dates <- dbGetQuery(con, 'SELECT 
    MIN(DATE) AS first_date, MAX(DATE) AS last_date FROM FARM_PRICES');

result_dates

first_date,last_date
<chr>,<chr>
1985-01-01,2020-12-01


7. Which crops have ever reached a farm price greater than or equal to $350 per metric tonne?





In [71]:
# Query to find crops with farm prices >= 350
high_price_crops <- dbGetQuery(con, 'SELECT DISTINCT CROP_TYPE 
  FROM FARM_PRICES WHERE PRICE_PRERMT >= 350');

high_price_crops

CROP_TYPE
<chr>
Canola


8. List the top 10 years of Wheat production in Saskatchewan in terms of harvested area.

In [ ]:
# Check column names in CROP_DATA
colnames(dbReadTable(con, "CROP_DATA"))

In [ ]:
dbGetQuery(con, "SELECT strftime('%Y', YEAR) AS TOP_10_YRS, GEO, HARVESTED_AREA
FROM CROP_DATA
WHERE CROP_TYPE= 'Wheat' AND
GEO='Saskatchewan'
ORDER BY HARVESTED_AREA DESC
LIMIT 10")

9. Rank the crop types harvested in Saskatchewan in the year 2000 by their average yield. Which crop performed best?

In [80]:
Saskatchewan_yield_2000 <- dbGetQuery(con, "
  SELECT CROP_TYPE, AVG_YIELD FROM CROP_DATA
  WHERE GEO = 'Saskatchewan' AND strftime('%Y', YEAR) = '2000'
  ORDER BY AVG_YIELD DESC");

Saskatchewan_yield_2000

CROP_TYPE,AVG_YIELD
<chr>,<int>
Barley,2800
Wheat,2200
Rye,2100
Canola,1400


10. Rank the crops and geographies by their average yield (KG per hectare) since the year 2000. Which crop and province had the highest average yield since the year 2000?

In [85]:
yield_ranking <- dbGetQuery(con, "
  SELECT CROP_TYPE, GEO, ROUND(AVG(AVG_YIELD), 2) AS AVG_YIELD
  FROM CROP_DATA
  WHERE strftime('%Y', YEAR) >= '2000'
  GROUP BY CROP_TYPE, GEO
  ORDER BY AVG_YIELD DESC");

head(yield_ranking, 10)

,CROP_TYPE,GEO,AVG_YIELD
,<chr>,<chr>,<dbl>
1,Barley,Alberta,3450.71
2,Barley,Canada,3253.76
3,Wheat,Alberta,3100.62
4,Barley,Saskatchewan,2971.05
5,Wheat,Canada,2845.33
6,Rye,Alberta,2683.81
7,Rye,Canada,2543.90
8,Wheat,Saskatchewan,2429.38
9,Rye,Saskatchewan,2226.71


11. How much wheat was harvested in Canada in the most recent year of the data?

In [86]:
wheat_recent_harvest <- dbGetQuery(con, "
  SELECT YEAR, SUM(HARVESTED_AREA) AS TOTAL_HARVESTED
  FROM CROP_DATA
  WHERE CROP_TYPE = 'Wheat' AND GEO = 'Canada'
    AND YEAR = (
      SELECT MAX(YEAR)
      FROM CROP_DATA
      WHERE CROP_TYPE = 'Wheat' AND GEO = 'Canada'
    )
  GROUP BY YEAR");

wheat_recent_harvest

YEAR,TOTAL_HARVESTED
<chr>,<int>
2020-12-31,10017800


12. What's the monthly price per metric tonne of Canola grown in Saskatchewan in both Canadian and US dollars?

In [88]:
dbListFields(con, "FARM_PRICES")

[1] "CD_ID"        "DATE"         "CROP_TYPE"    "GEO"          "PRICE_PRERMT"

In [90]:
dbListFields(con, "MONTHLY_FX")

[1] "DFX_ID"   "DATE"     "FXUSDCAD"

In [92]:
canola_prices <- dbGetQuery(con, "
  SELECT F.DATE, F.GEO, F.CROP_TYPE, F.PRICE_PRERMT AS CAD_PRICE,
    ROUND(F.PRICE_PRERMT / M.FXUSDCAD, 2) AS USD_PRICE
  FROM FARM_PRICES F
  JOIN MONTHLY_FX M ON F.DATE = M.DATE
  WHERE F.CROP_TYPE = 'Canola' AND F.GEO = 'Saskatchewan'
  ORDER BY F.DATE DESC LIMIT 6");

canola_prices;

DATE,GEO,CROP_TYPE,CAD_PRICE,USD_PRICE
<chr>,<chr>,<chr>,<dbl>,<dbl>
2020-12-01,Saskatchewan,Canola,507.33,396.11
2020-11-01,Saskatchewan,Canola,495.64,379.27
2020-10-01,Saskatchewan,Canola,474.80,359.30
2020-09-01,Saskatchewan,Canola,463.52,350.41
2020-08-01,Saskatchewan,Canola,464.60,351.38
2020-07-01,Saskatchewan,Canola,462.88,342.91


13. How many years did Barley yield at least 200 Kg per hectare in Canada?

In [96]:
dbGetQuery(con, "
  SELECT COUNT(DISTINCT YEAR) AS BLY_YRS_ABOVE_2MTPH FROM CROP_DATA
  WHERE AVG_YIELD > 2000 
  AND CROP_TYPE = 'Barley' AND GEO = 'Canada'");

BLY_YRS_ABOVE_2MTPH
<int>
52


14. How much farm land was seeded with Barley in Alberta but not harvested each year since the year 2000?

In [97]:
dbGetQuery(con, "
  SELECT 
    strftime('%Y', YEAR) AS YEAR, GEO, CROP_TYPE,SEEDED_AREA, 
    HARVESTED_AREA,
    100 * (SEEDED_AREA - HARVESTED_AREA) / SEEDED_AREA AS PCT_UNHARVESTED_AREA
  FROM CROP_DATA 
  WHERE 
    YEAR >= 2000 AND GEO = 'Alberta' AND CROP_TYPE = 'Barley'");

YEAR,GEO,CROP_TYPE,SEEDED_AREA,HARVESTED_AREA,PCT_UNHARVESTED_AREA
<chr>,<chr>,<chr>,<int>,<int>,<int>
2000,Alberta,Barley,2185300,1740100,20
2001,Alberta,Barley,1983000,1618700,18
2002,Alberta,Barley,2169100,1127000,48
2003,Alberta,Barley,2144800,1788700,16
2004,Alberta,Barley,1910100,1598500,16
2005,Alberta,Barley,1709800,1456900,14
2006,Alberta,Barley,1657100,1375900,16
2007,Alberta,Barley,1962700,1728000,11
2008,Alberta,Barley,1679400,1517600,9


15. Over the last 3 calendar years of data, what was the average value of the Canadian dollar relative to the USD?

In [98]:
dbGetQuery(con, "
  SELECT 
    MIN(DATE) AS AS_OF_DATE, 
    ROUND(AVG(FXUSDCAD), 4) AS FX_DAILY_AVG_CAD
  FROM DAILY_FX
  WHERE DATE >= DATE((SELECT MAX(DATE) FROM DAILY_FX), '-3 years')
");

AS_OF_DATE,FX_DAILY_AVG_CAD
<chr>,<dbl>
2018-02-20,1.3227


16. Create a view of the crop data with an FX column included

In [105]:
dbGetQuery (con, "SELECT CD_ID, YEAR, CROP_TYPE, GEO, SEEDED_AREA, HARVESTED_AREA, PRODUCTION, AVG_YIELD, FXUSDCAD
FROM CROP_DATA, DAILY_FX
WHERE strftime('%Y', CROP_DATA. YEAR) = strftime('%Y', DAILY_FX.DATE) and strftime('%m',
CROP_DATA. YEAR) = strftime('%m', DAILY_FX.DATE) LIMIT 5")

CD_ID,YEAR,CROP_TYPE,GEO,SEEDED_AREA,HARVESTED_AREA,PRODUCTION,AVG_YIELD,FXUSDCAD
<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>
624,2017-12-31,Barley,Alberta,1153400,1011700,3906000,3900,1.2729
624,2017-12-31,Barley,Alberta,1153400,1011700,3906000,3900,1.2695
624,2017-12-31,Barley,Alberta,1153400,1011700,3906000,3900,1.2680
624,2017-12-31,Barley,Alberta,1153400,1011700,3906000,3900,1.2757
624,2017-12-31,Barley,Alberta,1153400,1011700,3906000,3900,1.2843


## IV. Key Findings

- Provinces Analyzed: Alberta, Saskatchewan, and Canada overall.
- High-Value Crops: Canola reached farm prices > $350.
- Top Yields: Wheat and Canola generally led in yield performance post-2000.
- Exchange Rate: Recent 3-year average USD to CAD ≈ 1.32 (varies by exact data).
- Unharvested Land: Alberta shows consistent percentage of seeded but not harvested Barley.
- Price Insights: CAD-to-USD conversion is vital for profitability estimates in USD terms.

## V. Recommendations

- Invest in High-Yield, High-Value Crops: Prioritize Canola and Wheat, especially in Saskatchewan and Alberta.
- Monitor Currency Fluctuations: Use monthly FX data to time large-scale purchases or contracts.
- Explore Yield Efficiency: Focus on crops consistently above 2000 kg/hectare post-2000.
- Data-Driven Contracts: Use the percentage of unharvested land to negotiate pricing margins.
- Scale Further with Time Series Forecasting: Predict yields & prices with models using R packages like forecast or prophet.